# Data Preprocessing - Behaviorally-Optimised Call Script Analysis

## 1. Necessary Imports

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

import sys
sys.path.append('../')
import config
from utils import save_processed_data, print_section_header

## 2. Data Loading and Preprocessing

In [2]:
print_section_header("Data Loading and Preprocessing")

def load_and_combine_data(excel_path, control_sheet, pilot_sheet):
    """Load both sheets and combine with treatment labels"""
    print("Loading Task 2 data...")
    
    control_data = pd.read_excel(excel_path, sheet_name=control_sheet)
    pilot_data = pd.read_excel(excel_path, sheet_name=pilot_sheet)
    
    control_data['treatment'] = 'control'
    pilot_data['treatment'] = 'pilot'
    
    combined_data = pd.concat([control_data, pilot_data], ignore_index=True)
    
    # # Remove the original treatment-control indicator column
    # combined_data.drop(columns=['COLUMN_4'], inplace=True)
    
    print(f"Data loaded: {len(combined_data)} responses ({len(control_data)} control, {len(pilot_data)} pilot)")
    return combined_data


DATA LOADING AND PREPROCESSING


In [3]:
df = load_and_combine_data(config.EXCEL_FILE, config.CONTROL_SHEET, config.PILOT_SHEET)

print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())
print("\nFirst few rows:")
df.head()

Loading Task 2 data...
Data loaded: 647 responses (380 control, 267 pilot)
Dataset shape: (647, 14)
Columns: ['COLUMN_4', 'VOLT_FLAG', 'SURVEY_ID', 'SCORE', 'LTR_COMMENT', 'PRIMARY_REASON', 'TO_CHAR', 'CONNECTION_TIME', 'SALES_PERSON_SAT', 'SALES_FRIENDLY_SAT', 'COMMINICATION_SAT', 'FIRST_BILL_SAT', 'AGENT_KNOWLEDGE', 'treatment']

First few rows:


,COLUMN_4,VOLT_FLAG,SURVEY_ID,SCORE,LTR_COMMENT,PRIMARY_REASON,TO_CHAR,CONNECTION_TIME,SALES_PERSON_SAT,SALES_FRIENDLY_SAT,COMMINICATION_SAT,FIRST_BILL_SAT,AGENT_KNOWLEDGE,treatment
0,control,NaN,351124494,10,Conservations with your staff over the phone w...,"Customer Service,Installation,Processes/Journe...",2023-02-01 00:00:00,10.0,10.0,10.0,10.0,10.0,10.0,control
1,control,yes,351727144,10,"I spoke to tony and he was lovely, he talked m...","Brand,Changing ProviderEquipment,General Servi...",2023-02-01 00:00:00,10.0,8.0,10.0,10.0,10.0,10.0,control
2,control,yes,351645749,10,NaN,NaN,2023-02-01 00:00:00,10.0,10.0,10.0,10.0,10.0,10.0,control
3,control,yes,351707395,10,"Very positive so far. Cheaper, more included a...","General,Pricing,UK Legacy",2023-02-01 00:00:00,10.0,5.0,10.0,10.0,10.0,10.0,control
4,control,NaN,351119086,10,They was on time. Done a good job. Was very po...,Customer Service,2023-02-01 00:00:00,10.0,9.0,5.0,10.0,10.0,10.0,control


### 2.1 Clean the `date` column

I need to remove February data as required in the brief. Also, I see there are some messy dates like "01/03/2company3". I took the assumption that it is the year 2023 and the data was misplaced by replacing "02" from 2023 with "company".

In [4]:
def convert_excel_date(excel_date):
    """Convert various date formats to datetime"""
    if pd.isna(excel_date):
        return None
    
    # Handle malicious text dates (like "01/03/2company3")
    if isinstance(excel_date, str):
        if "company" in excel_date.lower():
            # Fix the malicious pattern: "01/03/2company3" -> "01/03/2023"
            fixed_date = excel_date.replace("company", "02")
            try:
                return pd.to_datetime(fixed_date, format='%m/%d/%Y')
            except:
                return None
        # Try to parse other date strings normally
        try:
            return pd.to_datetime(excel_date, errors='coerce')
        except:
            return None
    
    # Handle datetime objects (already converted by pandas)
    if isinstance(excel_date, pd.Timestamp):
        return excel_date
    
    # Handle normal Excel serial numbers
    if isinstance(excel_date, (int, float)):
        try:
            return datetime(1900, 1, 1) + pd.Timedelta(days=excel_date-2)
        except:
            return None
    
    # Try direct conversion as fallback
    try:
        return pd.to_datetime(excel_date, errors='coerce')
    except:
        return None

In [5]:
def filter_february_data(df):
    """Exclude February responses as required by brief"""
    print("Applying temporal filtering...")
    
    df['date_converted'] = df['TO_CHAR'].apply(convert_excel_date)
    df['month'] = df['date_converted'].dt.month
    
    initial_count = len(df)
    df_filtered = df[df['month'] != config.EXCLUDE_MONTH]
    final_count = len(df_filtered)
    
    print(f"Excluded {initial_count - final_count} February responses")
    print(f"Remaining: {final_count} responses")
    
    return df_filtered

In [6]:
df_filtered = filter_february_data(df)

print("\nDate range after filtering:")
print("Earliest:", df_filtered['date_converted'].min())
print("Latest:", df_filtered['date_converted'].max())
print("Valid months present:", sorted(df_filtered['month'].dropna().unique()))

Applying temporal filtering...
Excluded 65 February responses
Remaining: 582 responses

Date range after filtering:
Earliest: 2023-01-03 00:00:00
Latest: 2023-06-01 00:00:00
Valid months present: [1, 3, 4, 5, 6]


### 2.2 Clean the text column `LTR_COMMENT`

In [7]:
# Download required NLTK data
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

def clean_text(text):
    """Enhanced text preprocessing using NLTK"""
    if pd.isna(text) or text == 'NULL':
        return None
    
    text = str(text).lower()
    
    # Remove URLs and email addresses
    text = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remove punctuation but keep apostrophes for contractions
    text = re.sub(r'[^\w\s\']', ' ', text)
    
    # Remove numbers unless they're part of words
    text = re.sub(r'\b\d+\b', '', text)
    
    # Simple tokenization (split by spaces instead of NLTK)
    tokens = text.split()
    
    # Remove stopwords and short words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    
    text = ' '.join(tokens)
    
    return text if len(text) > 5 else None

In [8]:
df_filtered['cleaned_comment'] = df_filtered['LTR_COMMENT'].apply(clean_text)

# Check text cleaning results
total_comments = len(df_filtered)
valid_comments = df_filtered['cleaned_comment'].notna().sum()
print(f"Valid comments after cleaning: {valid_comments}/{total_comments}")

Valid comments after cleaning: 422/582


### 2.3 Create the VOLT and Non-VOLT segments

By my understanding of data, I infer and assume VOLT is a "premium" user base, so I'll go with that assumption for the rest of the analysis

In [9]:
def create_segments(df):
    """Create VOLT and Non-VOLT segments for each treatment"""
    df['segment'] = df['VOLT_FLAG'].apply(lambda x: 'VOLT' if x == 'yes' else 'Non-VOLT')
    df['group'] = df['segment'] + '_' + df['treatment']
    return df

## 3. Final Data Save and Export

In [10]:
# Create segments
df_final = create_segments(df_filtered)

# Data quality checks
print("Final dataset summary:")
print(f"Total responses: {len(df_final)}")
print(f"Comments with text: {df_final['cleaned_comment'].notna().sum()}")
print(f"Empty comments: {df_final['cleaned_comment'].isna().sum()}")

print(f"\nSegment distribution:")
print(df_final['group'].value_counts())

# Keep only essential columns
columns_to_keep = [
    'treatment', 'segment', 'group', 'VOLT_FLAG', 'SCORE', 
    'cleaned_comment', 'date_converted', 'month',
    'SALES_PERSON_SAT', 'SALES_FRIENDLY_SAT', 'COMMINICATION_SAT', 
    'FIRST_BILL_SAT', 'AGENT_KNOWLEDGE'
]

df_final = df_final[columns_to_keep]
print(f"\nFinal columns: {df_final.columns.tolist()}")

# Save for next notebook
save_processed_data(df_final, "cleaned_data.csv")

Final dataset summary:
Total responses: 582
Comments with text: 422
Empty comments: 160

Segment distribution:
group
Non-VOLT_control    177
Non-VOLT_pilot      164
VOLT_control        158
VOLT_pilot           83
Name: count, dtype: int64

Final columns: ['treatment', 'segment', 'group', 'VOLT_FLAG', 'SCORE', 'cleaned_comment', 'date_converted', 'month', 'SALES_PERSON_SAT', 'SALES_FRIENDLY_SAT', 'COMMINICATION_SAT', 'FIRST_BILL_SAT', 'AGENT_KNOWLEDGE']
Saved processed data: ../data/processed/cleaned_data.csv
